In this example, the Hamiltonian $H$ is a simple Pauli $X$ term, meaning the qubit's energy landscape is dominated by a field in the X direction: $H$. We want to simulate the time evolution of an initial state $|\psi (0)\rangle$ under this Hamiltonian for a specific time \(t\), which is governed by the unitary operator $U(t)=e^{-iHt/h}$.

Since complex exponential operators cannot always be directly mapped to a simple sequence of gates, Qiskit uses approximation methods like the Suzuki-Trotter decomposition to break down the large evolution into a series of smaller, implementable gates.

**PauliEvolutionGate**: This built-in Qiskit component automatically generates a quantum circuit that approximates the time evolution of a given SparsePauliOp.

**qiskit_dynamics**: For more complex, time-dependent, or open quantum systems (Lindblad dynamics), the more advanced qiskit_dynamics module provides dedicated solvers for numerical simulation.

In [ ]:
!pip install qiskit
!pip install qiskit_aer

In [9]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import PauliEvolutionGate
from qiskit_aer import AerSimulator
from qiskit_aer import Aer
import matplotlib.pyplot as plt

# 1. Define the Hamiltonian
# In Qiskit, Hamiltonians are often represented as a SparsePauliOp
# This H represents H = 1.0 * X
hamiltonian = SparsePauliOp.from_list([('X', 1.0)])
print(f"Hamiltonian: {hamiltonian}")

# 2. Define the initial state and time parameters
# Start with the qubit in the |0> state
initial_state = np.array([1, 0])
# Set the time duration for the simulation
time =np.pi #We choose pi for this example

# 3. Create the time evolution gate
# The PauliEvolutionGate creates a circuit that approximates the unitary e^(-iHt)
# By default, it uses the Lie-Trotter decomposition with a single step (order=1)
evolution_gate = PauliEvolutionGate(hamiltonian, time=time)

# 4. Build the quantum circuit
qc = QuantumCircuit(1)
# Apply the evolution gate to the circuit
qc.append(evolution_gate, [0])
qc.save_statevector()


# 5. Simulate the circuit
# Use the statevector simulator to get the final quantum state
simulator = AerSimulator(method="statevector")

#Transpile the circuit
t_qc = transpile(qc, simulator)

# Compile the circuit for the simulator
compiled_circuit = simulator.run(t_qc)
result = compiled_circuit.result()
final_state = result.get_statevector()

print(f"Initial state: |0> (represented as {initial_state})")
print(f"Final state after time t={time}: {final_state.data}")

# Expected result for H=X, t=pi: Initial state |0> evolves to -i|1> (or a phase equivalent)
# The final state should be close to [0, -1j]

Hamiltonian: SparsePauliOp(['X'],
              coeffs=[1.+0.j])
Initial state: |0> (represented as [1 0])
Final state after time t=3.141592653589793: [-1.+1.2246468e-16j -0.+0.0000000e+00j]
